In [ ]:
using LinearAlgebra, Optim

# Define the objective function
function f(x)
    return x' * A * x + b' * x
end

# Define the Hessian matrix
A = [2.0 1.0; 1.0 2.0]

# Define the gradient vector
b = [-1.0, -2.0]

# Define the initial guess
x0 = [0.0, 0.0]

# Compute the eigenvalues and eigenvectors of the Hessian matrix
vals, vecs = eigen(A)

# Use the GLTR algorithm to solve the trust-region subproblem
#result = trust_region(f, x0, vals, vecs)

# Use the IRRA algorithm to solve the trust-region subproblem
result_irra = irra(f, x0, vals, vecs)

# Use the IRA algorithm to solve the trust-region subproblem
result_ira = ira(f, x0, vals, vecs)

# Print the results
println("CG: $(result_cg.minimum) $(result_cg.iterations)")
println("BFGS: $(result_bfgs.minimum) $(result_bfgs.iterations)")
#println("GLTR: $(result_gltr.minimum) $(result_gltr.iterations)")
println("IRRA: $(result_irra.minimum) $(result_irra.iterations)")
println("IRA: $(result_ira.minimum) $(result_ira.iterations)")

In [ ]:
] add TrustOptim

In [ ]:
using LinearAlgebra, Optim

# Define the objective function
function f(x)
    return x' * A * x + b' * x
end

# Define the Hessian matrix
A = [2.0 1.0; 1.0 2.0]

# Define the gradient vector
b = [-1.0, -2.0]

# Define the initial guess
x0 = [0.0, 0.0]

# Compute the eigenvalues and eigenvectors of the Hessian matrix
vals, vecs = eigen(A)

# Define the GLTR algorithm
function gltr(f, x0, vals, vecs, Δ)
    n = length(x0)
    x = copy(x0)
    r = zeros(n)
    z = zeros(n)
    d = zeros(n)
    γ = Inf
    k = 0
    while γ > Δ && k < n
        k += 1
        r = Optim.gradient(f, x)
        z = vecs' * r
        d = zeros(n)
        for i in 1:k
            α = z[i] / vals[i]
            d += α * vecs[:, i]
            z -= α * vals[i] * vecs[:, i]
        end
        γ = norm(d)
        if γ <= Δ
            break
        end
        res = optimize(t -> f(x + t*d), 0.0, Δ, method = Newton())
        x += res.minimizer * d
    end
    return x
end

# Define the trust-region radius
Δ = 1.0

# Solve the trust-region subproblem using the GLTR algorithm
x_opt = gltr(f, x0, vals, vecs, Δ)

# Print the optimal solution
println("x_opt = $x_opt")

In [6]:
using LinearAlgebra, SparseArrays

function ira(A::SparseMatrixCSC{Float64, Int64}, m, maxiter, tol)
    # Initialize variables
    n = size(A, 1)
    V = spzeros(n, m)
    H = zeros(m, m - 1)
    beta = norm(A[:, 1])
    V[:, 1] = A[:, 1] / beta
    mu = Inf
    converged = false
    
    # IRA loop
    for j = 1:maxiter
        # Arnoldi iteration
        for i = 1:m - 1
            w = A * V[:, i]
            for k = 1:i
                H[k, i] = dot(V[:, k], w)
                w -= H[k, i] * V[:, k]
            end
            H[i + 1, i] = norm(w)
            if H[i + 1, i] < tol
                break
            end
            V[:, i + 1] = w / H[i + 1, i]
        end
        
        # Compute approximate eigenvalue and check for convergence
        T = H[1:m-1, :]
        Lambda, X = eigen(T)
        idx = argmax(abs(Lambda))
        mu_old = mu
        mu = beta * Lambda[idx]
        if abs(mu - mu_old) < tol
            converged = true
            break
        end
        
        # Restart with new initial vector
        V[:, 1] = X[:, idx]
        for i = 2:m
            V[:, i] = A * V[:, i - 1]
            for k = 1:i - 1
                H[k, i - 1] = dot(V[:, k], V[:, i])
                V[:, i] -= H[k, i - 1] * V[:, k]
            end
            H[i, i - 1] = norm(V[:, i])
            if H[i, i - 1] < tol
                break
            end
            V[:, i] /= H[i, i - 1]
        end
        beta = norm(V[:, m])
    end
    
    # Compute final eigenvector
    if converged
        return mu, V[:, m]
    else
        error("IRA failed to converge")
    end
end

ira (generic function with 2 methods)

In [7]:
# Generate a sparse matrix
using SparseArrays
n = 1000
A = sprandn(n, n, 0.1)

# Compute the largest eigenvalue using IRRA
using BenchmarkTools
@btime ira(A, 20, 5, 100, 1e-6)

# Check the result using the built-in eigvals function
using LinearAlgebra
λ = maximum(eigvals(A))
@show λ

LoadError: MethodError: no method matching ira(::SparseMatrixCSC{Float64, Int64}, ::Int64, ::Int64, ::Int64, ::Float64)
[0mClosest candidates are:
[0m  ira(::SparseMatrixCSC{Float64, Int64}, ::Any, ::Any, ::Any) at In[6]:3
[0m  ira(::SparseMatrixCSC, ::Any, ::Any, ::Any) at In[1]:3

In [ ]:
A = sprandn(10, 10, 0.1)

In [9]:
function irra(A::SparseMatrixCSC{T, Int64}, k::Int, m::Int, maxiter::Int, tol::T) where T<:AbstractFloat
    n = size(A, 1)
    Q = Matrix{T}(undef, n, k+m)
    H = Matrix{T}(undef, k+m, k)
    V = Matrix{T}(undef, n, m)
    W = Matrix{T}(undef, n, m)
    norms = similar(diag(A))
    R = SymTridiagonal{T}(undef, k)
    Q[:, 1] = randn(n)
    Q[:, 1] /= norm(Q[:, 1])
    λ_old = zero(T)
    converged = false
    for iter = 1:maxiter
        # Run the Arnoldi process
        for j = 1:k+m
            v = Q[:, j]
            w = A * v
            for i = 1:j
                H[i, j] = dot(Q[:, i], w)
                w -= H[i, j] * Q[:, i]
            end
            H[j+1, j] = norm(w)
            if H[j+1, j] == 0
                break
            end
            Q[:, j+1] = w / H[j+1, j]
        end

        # Compute the Ritz values and vectors
        T = H[1:k, 1:k]
        eigvals!(R, T)
        V[:, 1:k] = Q[:, 1:k] * R

        # Compute the refined vectors
        W[:, 1:m] = V[:, 1:k] * diagm(R)
        for j = k+1:k+m
            w = Q[:, j]
            for i = 1:k
                w -= dot(W[:, i], w) * W[:, i]
            end
            W[:, j-k] = w / norm(w)
        end
        V[:, k+1:k+m] = W

        # Compute the Ritz values and vectors of the refined subspace
        T = V' * A * V
        eigvals!(R, T)
        λ = maximum(R)

        # Check for convergence
        if abs(λ - λ_old) < tol
            converged = true
            break
        end
        λ_old = λ
    end

    # Return the largest Ritz value and corresponding Ritz vector
    v = V * R[1, 1]
    return λ, v
end

LoadError: syntax: local variable name "T" conflicts with a static parameter